# Importing and Installing Package

In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 816.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -q datasets
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
##Importing Packages
import re
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Loading and Data Preparation

In [5]:
dataset = load_dataset("HHousen/ParaSCI")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 338717
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 6433
    })
    test: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 4894
    })
})

In [6]:
train_data = dataset['train']
train_data

Dataset({
    features: ['sentence1', 'sentence2'],
    num_rows: 338717
})

In [7]:
def transform_data(data):
  return {
      'text': f"### Question: Repharse this scientific text: {data['sentence1']}\n ### Answer: {data['sentence2']}"
  }

In [8]:
transform_data(train_data[10])

{'text': '### Question: Repharse this scientific text: shared task is a new approach to time normalization based on the semantically compositional annotation of time expressions .\n ### Answer: the parsing time normalization task is the first effort to extend time normalization to richer and more complex time expressions .'}

In [9]:
transformed_dataset = train_data.map(transform_data)

Map:   0%|          | 0/338717 [00:00<?, ? examples/s]

In [10]:
transformed_dataset = transformed_dataset.remove_columns(['sentence1','sentence2'])
transformed_dataset = transformed_dataset[:500]

In [16]:
import pandas as pd
transformed_dataset = pd.DataFrame.from_dict(transformed_dataset)

In [20]:
from datasets import Dataset
transformed_dataset = Dataset.from_pandas(transformed_dataset)

In [22]:
transformed_dataset

Dataset({
    features: ['text'],
    num_rows: 500
})

# Fine-Tuning LLM

In [23]:
##The model -> Fine-Tunning
model_name = "NousResearch/Llama-2-7b-chat-hf"
##Fine-tuned model name
new_model = "ParaSCI_Llama"

# QLoRA parameters

In [24]:
##LoRA attention dimension
lora_r = 64

##Alpha parameter for LoRA scaling
lora_alpha = 16

##Dropout probability for LoRA layers
lora_dropout = 0.1

# BitsandBytes parameters

In [25]:
##Activate 4-bit precision base model loading
use_4bit = True

##Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

##Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

##Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Training Arguments parameters

In [26]:
##Output directory where the model predictions and checkpoints will be stored
output_dir = "./ParaSCI/"

##Number of training epochs
num_train_epochs = 1

##Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

##Batch size per GPU for training
per_device_train_batch_size = 4

##Batch size per GPU for evaluation
per_device_eval_batch_size = 4

##Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

##Enable gradient checkpointing
gradient_checkpointing = True

##Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

##Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

##Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

##Optimizer to use
optim = "paged_adamw_32bit"

##Learning rate schedule
lr_scheduler_type = "cosine"

##Number of training steps (overrides num_train_epochs)
max_steps = -1

##Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

##Save checkpoint every X updates steps
save_steps = 0

##Log every X updates steps
logging_steps = 20

# SFT Parameters

In [27]:
##Maximum sequence length to use
max_seq_length = None

##Pack multiple short examples in the same input sequence to increase efficiency
packing = False

##Load the entire model on the GPU 0
device_map = {"": 0}

# Load tokenizer and model with QLoRA configuration

In [28]:
##Loading dataset
dataset = transformed_dataset

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

##Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model

In [29]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

# Load LLaMA tokenizer

In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [31]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters

In [32]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

# Model Training

In [33]:
##Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
20,3.277200
40,2.234200
60,1.740600
80,1.823800
100,1.727400
120,1.688900


TrainOutput(global_step=125, training_loss=2.069271987915039, metrics={'train_runtime': 173.3919, 'train_samples_per_second': 2.884, 'train_steps_per_second': 0.721, 'total_flos': 674998323609600.0, 'train_loss': 2.069271987915039, 'epoch': 1.0})

In [34]:
##Save trained model
trainer.model.save_pretrained(new_model)

# Making the Model to the Local

In [35]:
import shutil
folder_to_zip = '/content/ParaSCI_Llama'
zip_filename = 'ParaSCI_llama'
shutil.make_archive(zip_filename, 'zip', folder_to_zip)

'/content/ParaSCI_llama.zip'

# Empty VRAM

In [36]:
del model
del trainer
import gc
gc.collect()
gc.collect()

20933

# Inference

In [37]:
##Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

##Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [38]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "repharse the following text: twitter is a very popular micro blogging site"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


In [39]:
print(result[0]['generated_text'])

<s>[INST] repharse the following text: twitter is a very popular micro blogging site [/INST]  Twitter is a very popular microblogging site that allows users to share short messages of up to 280 characters.

In this context, "microblogging" refers to the practice of creating and sharing short blog-like posts, typically on a social media platform. Twitter is one of the most well-known and widely-used microblogging platforms, with millions of users sharing their thoughts, opinions, and experiences in the form of tweets.
